In [1]:
import pandas as pd
from pyproj import Proj, transform
import fiona
from fiona.crs import from_epsg
import logging
import sys
from aabbtree import AABB, AABBTree
import math


# outProj = Proj(init='epsg:3857')
# inProj = Proj(init='epsg:4326') # It is equal to WSG

In [2]:
# Parametros

espacamento = 50
n = 20 # multiplicador da area de amortização

In [3]:
# Carregar base em metros da area de preservação do chico mendes
chico_mendes = fiona.open("Mapas_sim/shapes_sim_meters/area_chico_mendes.shp")
print (chico_mendes.schema)

{'properties': OrderedDict([('id', 'float:19'), ('Unidade_de', 'str:254'), ('Categoria', 'str:254'), ('Ano_de_Cri', 'int:9'), ('Zoneamento', 'str:254'), ('Área_hecta', 'float:33.15'), ('Jurisdição', 'str:254'), ('Plano_de_M', 'str:254')]), 'geometry': 'Polygon'}


In [4]:
# Estrutura dos arquivos com dados
chico_mendes[0]

{'type': 'Feature',
 'id': '0',
 'properties': OrderedDict([('id', 897.0),
              ('Unidade_de', 'PARQUE NATURAL MUNICIPAL CHICO MENDES'),
              ('Categoria', 'PROTEÇÃO INTEGRAL'),
              ('Ano_de_Cri', None),
              ('Zoneamento', None),
              ('Área_hecta', 40.65),
              ('Jurisdição', 'MUNICIPAL'),
              ('Plano_de_M', None)]),
 'geometry': {'type': 'Polygon',
  'coordinates': [[(-4838947.215019874, -2634641.6353100897),
    (-4838947.215019874, -2634641.6353100897),
    (-4838911.530555228, -2634641.478797528),
    (-4838880.228516291, -2634641.665790127),
    (-4838865.3888489315, -2634642.0394124887),
    (-4838850.586696242, -2634643.1607634327),
    (-4838835.858904972, -2634645.0269402442),
    (-4838821.242655828, -2634647.6333469907),
    (-4838806.774795566, -2634650.973331637),
    (-4838792.491503024, -2634655.038669846),
    (-4838778.428845712, -2634659.8189602373),
    (-4838764.622223226, -2634665.3021081625),
    (

In [5]:
listaCoordenadas = chico_mendes[0]['geometry']['coordinates']

In [6]:
type(listaCoordenadas)

list

In [7]:
# retirar ponto x máximo e ponto x mínimo, referente a projeção 3857
# analogo para o ymin e ymax

xmin = 999999999999999
xmax = -99999999999999
ymax = xmax
ymin = xmin
for elem in listaCoordenadas[0]:
    
    if elem[0] > xmax:
        xmax = elem[0]
    elif elem[0] < xmin:
        xmin = elem[0]
        
    if elem[1] > ymax:
        ymax = elem[1]
    elif elem[1] < ymin:
        ymin = elem[1]

In [8]:
print('X Max: ', xmax, '\nX Min: ', xmin)
print('YMax: ', ymax, '\nY Min: ', ymin)

X Max:  -4838636.088731658 
X Min:  -4839587.213704263
YMax:  -2634641.478797528 
Y Min:  -2635360.089792147


In [9]:
yminAmortizado = ymin + (-n*espacamento) 
ymaxAmortizado = ymax + (n*espacamento) 
xminAmortizado = xmin + (-n*espacamento) 
xmaxAmortizado = xmax + (n*espacamento)

In [10]:
print('X Max: ', xmaxAmortizado, '\nX Min: ', xminAmortizado)
print('YMax: ', ymaxAmortizado, '\nY Min: ', yminAmortizado)

X Max:  -4837636.088731658 
X Min:  -4840587.213704263
YMax:  -2633641.478797528 
Y Min:  -2636360.089792147


In [11]:
tree = AABBTree()

treeEstudo = AABBTree()
# (xmin, xmax), (ymin, ymax)
area_estudo = AABB([(xmin, xmax), (ymin, ymax)])

area_amortizada = AABB([(xminAmortizado, xmaxAmortizado), (yminAmortizado, ymaxAmortizado)])


In [12]:
# aabb2 = AABB([(10,15), (10,15)])

tree.add(area_amortizada, 'envol')
treeEstudo.add(area_estudo)
a = tree.does_overlap(area_estudo)

print(a)


True


In [13]:
escolas = fiona.open("Mapas_sim/shapes_sim_meters/escola_metros.shp")
print (escolas.schema)

{'properties': OrderedDict([('INEP', 'int:9'), ('Escola', 'str:254'), ('Rede', 'str:254'), ('Localizaçã', 'str:254'), ('Município', 'str:254'), ('Bairro', 'str:254'), ('Endereço', 'str:254'), ('Telefone', 'str:254'), ('Total_Matr', 'float:19'), ('Infantil', 'float:19'), ('Creche', 'float:19'), ('PreEscola', 'float:19'), ('Fundamenta', 'float:19'), ('Fundament0', 'float:19'), ('Fundament1', 'float:19'), ('Ensino_Med', 'float:19'), ('Profission', 'int:9'), ('EJA', 'int:9'), ('Especial', 'int:9'), ('Latitude', 'float:33.15'), ('Longitude', 'float:33.15')]), 'geometry': 'Point'}


In [14]:
escolas[0]['geometry']

{'type': 'Point', 'coordinates': (-4809884.843755053, -2627191.2669907277)}

In [15]:
a,b = escolas[0]['geometry']['coordinates']


In [16]:
a

-4809884.843755053

In [17]:
listaEscolas = []

for escola in escolas:
    x, y = escola['geometry']['coordinates']
    
    t = AABB([(x, x), (y, y)])
    
    if(tree.does_overlap(t)):
        listaEscolas.append(escola)
    

In [18]:
sink_schema = escolas.schema.copy()
with fiona.open(
        'escola_estudo.shp', 'w',
        crs=from_epsg(3857),
        driver=escolas.driver,
        schema=sink_schema,
        ) as sink:
    
    for e in listaEscolas:
        sink.write(e)

In [19]:
saude = fiona.open("Mapas_sim/shapes_sim_meters/saude_metros.shp")
print (saude.schema)

{'properties': OrderedDict([('Município', 'str:254'), ('Código_CNE', 'str:254'), ('Razão_Soci', 'str:254'), ('Estabeleci', 'str:254'), ('CNPJ_Mante', 'str:254'), ('Endereço', 'str:254'), ('Telefone', 'str:254'), ('Email_ou_F', 'str:254'), ('Tipo', 'str:254'), ('Gestão', 'str:254')]), 'geometry': 'Point'}


In [20]:
listaSaude = []

for unidade in saude:
    x, y = unidade['geometry']['coordinates']
    
    t = AABB([(x, x), (y, y)])
    
    if(tree.does_overlap(t)):
        listaSaude.append(unidade)
    

In [21]:
listaSaude

[{'type': 'Feature',
  'id': '1315',
  'properties': OrderedDict([('Município', 'RIO DE JANEIRO'),
               ('Código_CNE', '3304557506759'),
               ('Razão_Soci',
                'SECRETARIA DE ESTADO DE SAUDE DO RIO DE JANEIRO'),
               ('Estabeleci', 'SAMU 192 PBAM01 RECREIO DOS BANDEIRANTES'),
               ('CNPJ_Mante', '42498717000155'),
               ('Endereço',
                'AV GLAUCIO GIL, S/N - RECREIO DOS BANDEIRA, RIO DE JANEIRO - RJ, 22790-000'),
               ('Telefone', None),
               ('Email_ou_F', None),
               ('Tipo',
                'UNIDADE MOVEL DE NIVEL PRE-HOSPITALAR NA AREA DE URGENCIA'),
               ('Gestão', 'ESTADUAL')]),
  'geometry': {'type': 'Point',
   'coordinates': (-4837878.411764581, -2634326.5910085365)}},
 {'type': 'Feature',
  'id': '1657',
  'properties': OrderedDict([('Município', 'RIO DE JANEIRO'),
               ('Código_CNE', '3304552708213'),
               ('Razão_Soci',
                'SMS 

In [22]:
sink_schema = saude.schema.copy()
with fiona.open(
        'saude_estudo.shp', 'w',
        crs=from_epsg(3857),
        driver=saude.driver,
        schema=sink_schema,
        ) as sink:
    
    for e in listaSaude:
        sink.write(e)

In [23]:
onibus = fiona.open("Mapas_sim/shapes_sim_meters/ponto_onibus_metros.shp")
print (onibus.schema)

{'properties': OrderedDict([('Nome_da_Li', 'str:254'), ('Número_da_', 'str:254'), ('Rota', 'str:254')]), 'geometry': 'MultiPoint'}


In [24]:
len(onibus)#['geometry']['coordinates'][176]


499

In [25]:
listaOnibus = []
i = 0
for linha in onibus:
    for ponto in linha['geometry']['coordinates']:

        x, y = ponto
        t = AABB([(x, x), (y, y)])

        if(tree.does_overlap(t)):
            listaOnibus.append(linha)
            break


In [26]:
len(listaOnibus)

28

In [27]:
sink_schema = onibus.schema.copy()
with fiona.open(
        'onibus_estudo.shp', 'w',
        crs=from_epsg(3857),
        driver=onibus.driver,
        schema=sink_schema,
        ) as sink:
    
    for e in listaOnibus:
        sink.write(e)

In [28]:
# xmin com ymax (0,0)
xminAmortizado

-4840587.213704263

In [29]:
xmaxAmortizado - xminAmortizado

2951.1249726051465

In [30]:
math.ceil( (ymaxAmortizado - yminAmortizado)/espacamento )

55

In [31]:
ymaxAmortizado - yminAmortizado

2718.6109946188517

In [32]:
escolas = fiona.open("chico_mendes/escola_estudo.shp")
saude = fiona.open("chico_mendes/saude_estudo.shp")
onibus = fiona.open("chico_mendes/onibus_estudo.shp")
lagoa = fiona.open("chico_mendes/lagoa_metros.shp")
chico_mendes = fiona.open("Mapas_sim/shapes_sim_meters/area_chico_mendes.shp")

In [33]:
lagoa[0]

{'type': 'Feature',
 'id': '0',
 'properties': OrderedDict([('id', 957.0),
              ('Classes',
               'AREAS DE PRESERVACAO PERMANENTE - LAGOS E LAGOAS URBANAS'),
              ('Perimetro_', 3.42628406312),
              ('Area_Hecta', 5.14088569943)]),
 'geometry': {'type': 'Polygon',
  'coordinates': [[(-4839352.668323951, -2635081.1750952885),
    (-4839383.96078941, -2635079.9233239028),
    (-4839388.011928322, -2635079.50530074),
    (-4839391.979800251, -2635078.5819243877),
    (-4839400.324420598, -2635076.078503056),
    (-4839404.443909677, -2635074.53123719),
    (-4839408.318718514, -2635072.440154537),
    (-4839411.878159233, -2635069.84335635),
    (-4839415.057666523, -2635066.7884994075),
    (-4839419.229921039, -2635062.1988367722),
    (-4839421.782254326, -2635059.0129863257),
    (-4839423.919699867, -2635055.532125798),
    (-4839425.609418418, -2635051.8105644295),
    (-4839428.198264499, -2635045.01479816),
    (-4839434.8159855865, -2635042.02

In [34]:
listaL = lagoa[0]['geometry']['coordinates']


In [35]:
nextX = espacamento
nextY = -espacamento

startX = xminAmortizado
startY = ymaxAmortizado
nowX = startX + nextX
nowY = startY + nextY

var = 0.000000001

idString = 'id'
xString = 'local_x'
yString = 'local_y'
escolaString = 'escola'
onibusString = 'linha_onibus'
saudeString = 'unidade_saude'
chicoString = 'area_parque_cm'
lagoaString = 'area_lagoa_cm'
ocupaString = 'ocupado'

dict_cm = {}

id_ = 0
for i in range(0, math.ceil( (xmaxAmortizado - xminAmortizado)/espacamento )):
    for j in range(0, math.ceil( (ymaxAmortizado - yminAmortizado)/espacamento )):
        
        cm = 0
        l_cm = 0
        ocupa = 1
        listaOnibus = []
        listaEscolas = []
        listaSaude = []
        
        tree = AABBTree()

        # (xmin, xmax), (ymin, ymax)
        area_estudo = AABB([(startX, nowX), (nowY, startY)])
        tree.add(area_estudo, 'envol')

        
        for linha in onibus:
            for ponto in linha['geometry']['coordinates']:
                x, y = ponto
                t = AABB([(x, x), (y, y)])
                if(tree.does_overlap(t)):
                    listaOnibus.append(linha)
#                     ocupa = 1
                    break
        
        for unidade in saude:
            x, y = unidade['geometry']['coordinates']

            t = AABB([(x, x), (y, y)])

            if(tree.does_overlap(t)):
#                 ocupa = 1
                listaSaude.append(unidade)
        
        
        for escola in escolas:
            x, y = escola['geometry']['coordinates']

            t = AABB([(x, x), (y, y)])

            if(tree.does_overlap(t)):
                listaEscolas.append(escola)
#                 ocupa = 1
                
        for coord in listaCoordenadas[0]:
            x, y = coord
            
            t = AABB([(x, x), (y, y)])

            if(tree.does_overlap(t)):
                cm = 1
                ocupa = 0
                
        for coordL in listaL[0]:
            x,y = coordL
            t = AABB([(x, x), (y, y)])

            if(tree.does_overlap(t)):
                l_cm = 1
                ocupa = 0
                
                
        dict_cm[id_] = {idString : id_, 'i': i, 'j': j, xString : startX, yString : startY, escolaString : listaEscolas, \
                     saudeString : listaSaude, onibusString : listaOnibus, chicoString : cm, lagoaString : l_cm, ocupaString : ocupa }
        
        
        id_ += 1
        startX = nowX + var
        nowX += nextX
        
    startX = xminAmortizado
    nowX = startX + nextX
    startY = nowY - var
    nowY += nextY
        
        

In [36]:
import csv

w = csv.writer(open("output.csv", "w"))
for key, val in dict_cm.items():
    w.writerow([key, val])

In [39]:
dict_cm[0]

{'id': 0,
 'i': 0,
 'j': 0,
 'local_x': -4840587.213704263,
 'local_y': -2633641.478797528,
 'escola': [],
 'unidade_saude': [],
 'linha_onibus': [],
 'area_parque_cm': 0,
 'area_lagoa_cm': 0}

In [ ]:
serie_historica = []
# serie_historica.append()
def ocupaEspaco(dict_cm, treeEstudo):
    
    tree = AABBTree()
    
    area_estudo = AABB([(startX, nowX), (nowY, startY)])